Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the book **Grammaire de Wolof Moderne** without considering the definitions. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup, T5ForConditionalGeneration,\
    get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v3 import SentenceDataset # v2 -> v3
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3_2.json")


In [3]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p, 
                                                          aug_word_max=max_len),
                                          remove_mark_space, delete_guillemet_space)

  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p, 
                                                          aug_word_max= max_len),
                                          remove_mark_space, delete_guillemet_space, end_mark_fn)
    
  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        cp1_transformer = fr_augmentation)

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure the model and the evaluation function ⚙️

Let us evaluate the predictions with the `bleu` metric.

In [4]:
%%writefile wolof-translate/wolof_translate/utils/evaluation.py
from tokenizers import Tokenizer
from typing import *
import numpy as np
import evaluate

class TranslationEvaluation:
    
    def __init__(self, 
                 tokenizer: Tokenizer,
                 decoder: Union[Callable, None] = None,
                 metric = evaluate.load('sacrebleu'),
                 ):
        
        self.tokenizer = tokenizer
        
        self.decoder = decoder
        
        self.metric = metric
    
    def postprocess_text(self, preds, labels):
        
        preds = [pred.strip() for pred in preds]
        
        labels = [[label.strip()] for label in labels]
        
        return preds, labels

    def compute_metrics(self, eval_preds):

        preds, labels = eval_preds

        if isinstance(preds, tuple):
        
            preds = preds[0]
        
        decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
        
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)

        result = self.metric.compute(predictions=decoded_preds, references=decoded_labels)
        
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != self.tokenizer.pad_token_id) for pred in preds]
        
        result["gen_len"] = np.mean(prediction_lens)
        
        result = {k: round(v, 4) for k, v in result.items()}
        
        return result

Overwriting wolof-translate/wolof_translate/utils/evaluation.py


Let us initialize the evaluation object.

In [5]:
%run wolof-translate/wolof_translate/utils/evaluation.py
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

In [6]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb v5

In [10]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.9833729160799256,
    'fr_word_p': 0.8864155155342781,
    'learning_rate': 0.005124817704092204,
    'weight_decay': 0.011812072968025469,
    'batch_size': 8,
    'warmup_ratio': 0.0,
    'max_epoch': 200,
    'max_len': 95,
    'end_mark': 4,
    'bleu': 1.3977,
    'model_dir': 'data/checkpoints/fw_t5_small_custom_train_v5_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_fw_v5/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 5, evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], 51,
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
# length = len(train_dataset)

# n_steps = length // config['batch_size']

# num_steps = config['max_epoch'] * n_steps

# warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# # Initialize the scheduler parameters
# scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                # lr_scheduler=get_linear_schedule_with_warmup,
                # lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_fw"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best=True)

        

### ---

In [8]:
trainer.train(epochs = config['max_epoch'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/197 [00:00<?, ?it/s]

For epoch 4: 


Test batch number 1:   0%|          | 0/11 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 11: 100%|██████████| 11/11 [00:04<00:00,  2.70batches/s]



Metrics: {'train_loss': 0.4865215807837784, 'test_loss': 0.7882578237490221, 'bleu': 0.9016, 'gen_len': 7.8512}




  1%|          | 1/197 [00:58<3:11:01, 58.48s/it]

...

 99%|█████████▉| 196/197 [3:34:25<08:49, 529.47s/it]

For epoch 200: 


Test batch number 11: 100%|██████████| 11/11 [00:03<00:00,  2.75batches/s]



Metrics: {'train_loss': 0.008034040725164155, 'test_loss': 1.1120836463841526, 'bleu': 4.7986, 'gen_len': 8.0655}




100%|██████████| 197/197 [3:35:15<00:00, 65.56s/it] 


### Predictions and Evaluation

In [11]:
# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        tokenizer,
                                        truncation = True)

Let us make the evaluation and print the predicted sentences.

In [12]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 1:   0%|          | 0/12 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Evaluation batch number 12: 100%|██████████| 12/12 [00:04<00:00,  2.60batches/s]


In [13]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
177,Tout autre mouton que tu vois.,Meneen xar mépp moo gis.,Bi ŋga dee dem
178,Ceci est une tasse et une assiette. On y verse...,Lii nag ëe kaas la ak palaat. Nu ciy xelli kaf...,"Ci biir nataal bi maa ngi ciy gis ay batã, bat..."
179,"Ça c'est un terrain de jeu, c'est-à-dire un tr...",Waaw lii ab fowu la maanaam stade fowu fowu la...,Waaw nataal bii nataal la boob ay nit ñu baree...
180,Quel être!,Moo di loola!,Demleen!
181,C'est moi qui ai été,Maa dem,Maa demoon
182,Il est par-là!,Mi ŋgi foofule!,Dem na ca subë.
183,Un homme n'a été,Góor demul,Demoon ba Ndar.
184,Surtout rentrez chez vous!,Te nag ŋgeen dem ñibbi!,Nataal bii ma gis
185,Surveille-moi ceux-là!,Seetal ma ñenn ñuu!,Kile la.
186,Alors celui-ci s'en alla!,Noona kooku dem!,Demleen foofu!


In [14]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
168,Il a vu l'autre.,Gis na keneen ki.,Gis na ma!
124,"Tout ce verbiage, c'est pour que tu ne viennes...","Wax ji yépp, bañ-ŋga-ñëw la.",Ŋga dem la ñooñu bëgg.
123,C'est l'autre endroit qui n'a pas de trou.,Feneen fi bëttul la.,Yaa demulwoon
89,Dis-lui qu'il ne vienne pas,Ni ka bu mu ñëw,Demal ndax mu ñëw ndax it mu génn!
177,Tout autre mouton que tu vois.,Meneen xar mépp moo gis.,Bi ŋga dee dem
18,C'est son ami!,Xaritam la!,Sa xarit la!
118,Un seul est arrivé,Menn doon na ñëw,Nit dem na!
59,Il y est.,Mi ŋgi fi.,Dem na.
30,"Te voilà debout, ici.",Yaa ŋgi tawax.,Yaa ka gis moom.
178,Ceci est une tasse et une assiette. On y verse...,Lii nag ëe kaas la ak palaat. Nu ciy xelli kaf...,"Ci biir nataal bi maa ngi ciy gis ay batã, bat..."
